<a href="https://colab.research.google.com/github/rajimuth/Big_Data/blob/main/PySPARK_tokenize_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nltk

In [19]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [20]:
import nltk
from nltk import word_tokenize
text = word_tokenize("I enjoy biking on the trails")
output = nltk.pos_tag(text)
print(output)

[('I', 'PRP'), ('enjoy', 'VBP'), ('biking', 'VBG'), ('on', 'IN'), ('the', 'DT'), ('trails', 'NNS')]


In [21]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version


In [22]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 https://developer.download.nvidia.com/c

In [23]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [24]:
from pyspark.ml.feature import Tokenizer

In [25]:
dataframe =spark.createDataFrame([(0,"Spark is great"),(1,"We are learning Spark"),(2,"Spark is better than Hadoop")],["id","sentence"])

In [26]:
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+



In [27]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_c86849cc2e9c

In [28]:
tokenized_df=tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)

+---+---------------------------+---------------------------------+
|id |sentence                   |words                            |
+---+---------------------------+---------------------------------+
|0  |Spark is great             |[spark, is, great]               |
|1  |We are learning Spark      |[we, are, learning, spark]       |
|2  |Spark is better than Hadoop|[spark, is, better, than, hadoop]|
+---+---------------------------+---------------------------------+



In [29]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [30]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [31]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [32]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer
tokenized_df=tokenizer.transform(dataframe)
tokenized_df.withColumn("tokens",count_tokens(col("words"))).show(truncate=False)

+---+---------------------------+---------------------------------+------+
|id |sentence                   |words                            |tokens|
+---+---------------------------+---------------------------------+------+
|0  |Spark is great             |[spark, is, great]               |3     |
|1  |We are learning Spark      |[we, are, learning, spark]       |4     |
|2  |Spark is better than Hadoop|[spark, is, better, than, hadoop]|5     |
+---+---------------------------+---------------------------------+------+

